In [1]:
import collections
import numpy as np
import pandas as pd
import re

from argparse import Namespace

In [2]:
args = Namespace(
    train_pos="data/imdb/train_pos.csv",
    train_neg="data/imdb/train_neg.csv",
    test_pos ="data/imdb/test_pos.csv", 
    test_neg ="data/imdb/test_neg.csv", 
    val_prop=0.1, # validation propotion
#     tr_pos = 12500,
#     tr_neg = 12500,
#     tt_pos = 12500,
#     tt_neg = 12500,
    tr_pos = 500,
    tr_neg = 500,
    tt_pos = 500,
    tt_neg = 500,
    output_csv="data/imdb/output.csv",
    seed=1337
)

In [3]:
train_pos = pd.read_csv(args.train_pos, header=None, names=['label', 'review'])
train_neg = pd.read_csv(args.train_neg, header=None, names=['label', 'review'])
test_pos = pd.read_csv(args.test_pos, header=None, names=['label', 'review'])
test_neg = pd.read_csv(args.test_neg, header=None, names=['label', 'review'])

In [4]:
"""
for steady performance, simply take the first n rows
"""

data_total= pd.DataFrame()

data_total = data_total.append(train_pos[0:args.tr_pos],ignore_index=True)
data_total = data_total.append(train_neg[0:args.tr_neg],ignore_index=True)

data_total = data_total.append(test_pos[0:args.tt_pos],ignore_index=True)
data_total = data_total.append(test_neg[0:args.tt_neg],ignore_index=True)



"""
assign splits to all data points, including splitting validation set
"""
data_total.loc[:,"split"] = "train"

data_total.iloc[args.tr_pos+args.tr_neg-1:,2] = "test"

data_total.iloc[int(args.tr_pos*(1-args.val_prop)):args.tr_pos,2] = "val"
data_total.iloc[int(args.tr_pos+args.tr_neg*(1-args.val_prop)):args.tr_pos+args.tr_neg,2] = "val"


In [5]:
# check the split value
data_total.split.value_counts()

test     1000
train     900
val       100
Name: split, dtype: int64

In [6]:
data_total.to_csv(args.output_csv, index=False)